### Systeme de recomendation python

In [1]:
import pandas as pd
# load data
metadata =  pd.read_csv("movies_metadata.csv", low_memory=False) #low_memory :Traitez en interne le fichier par blocs

# print first three rows
metadata.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [2]:
# Calculate mean of vote average column
C = metadata['vote_average'].mean()
print(C)

5.618207215134185


In [3]:
# Calculate the minimum number of votes required to be in the chart, m
m = metadata['vote_count'].quantile(0.90)
print(m)

160.0


In [4]:
# Filter out all qualified movies into a new DataFrame
q_movies = metadata.copy().loc[metadata['vote_count'] >= m]
q_movies.shape

(4555, 24)

In [5]:
metadata.shape

(45466, 24)

In [6]:
# Function that computes the weighted rating of each movie
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [7]:
# Define a new feature 'score' and calculate its value with `weighted_rating()`
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [8]:
#Sort movies based on score calculated above
q_movies = q_movies.sort_values('score', ascending=False)

#Print the top 15 movies
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(20)

,title,vote_count,vote_average,score
314,The Shawshank Redemption,8358.0,8.5,8.445869
834,The Godfather,6024.0,8.5,8.425439
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.421453
12481,The Dark Knight,12269.0,8.3,8.265477
2843,Fight Club,9678.0,8.3,8.256385
292,Pulp Fiction,8670.0,8.3,8.251406
522,Schindler's List,4436.0,8.3,8.206639
23673,Whiplash,4376.0,8.3,8.205404
5481,Spirited Away,3968.0,8.3,8.196055
2211,Life Is Beautiful,3643.0,8.3,8.187171


In [9]:
# Rcommender basé sur le contenu 

#Print plot overviews of the first 5 movies.
metadata['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

In [10]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
metadata['overview'] = metadata['overview'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(metadata['overview'])
#Output the shape of tfidf_matrix
tfidf_matrix.shape

(45466, 75827)

In [11]:
#Array mapping from feature integer indices to feature name.
tfidf.get_feature_names()[5000:5010]

['avails',
 'avaks',
 'avalanche',
 'avalanches',
 'avallone',
 'avalon',
 'avant',
 'avanthika',
 'avanti',
 'avaracious']

In [12]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix[0])
print(tfidf_matrix[0].shape)
print(tfidf_matrix.shape)

(1, 75827)
(45466, 75827)


In [13]:
cosine_sim.shape

(45466, 1)

In [14]:
cosine_sim[0]

array([1.])

In [15]:
cosine_sim[1]

array([0.01504121])

In [16]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(metadata.index, index=metadata['title']).drop_duplicates()

In [17]:
indices[:10]

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
Heat                           5
Sabrina                        6
Tom and Huck                   7
Sudden Death                   8
GoldenEye                      9
dtype: int64

In [18]:
# Function that takes in movie title as input and outputs most similar movies
import numpy as np
from time import time
from sklearn.metrics.pairwise import linear_kernel 

def get_recommendations(title):
    start = time()
    # Get the index of the movie that matches the title
    idx = indices[title]
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix[idx])
    #print(cosine_sim.shape)
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse= True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[:10]
    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]
    end = time()
    print('time :' ,end - start)
    print("__________________________________________________________")
    # Return the top 10 most similar movies
    return metadata['title'].iloc[movie_indices]

In [19]:
get_recommendations('The Dark Knight Rises')

time : 1.1738340854644775
__________________________________________________________


18252                                The Dark Knight Rises
12481                                      The Dark Knight
150                                         Batman Forever
1328                                        Batman Returns
15511                           Batman: Under the Red Hood
585                                                 Batman
21194    Batman Unmasked: The Psychology of the Dark Kn...
9230                    Batman Beyond: Return of the Joker
18035                                     Batman: Year One
19792              Batman: The Dark Knight Returns, Part 1
Name: title, dtype: object

In [20]:
# utiliser PCA sur tfidf
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=500)
svd_matrix = svd.fit_transform(tfidf_matrix)

In [21]:
svd_matrix.shape

svd_matrix[0].reshape(1, -1).shape

(1, 500)

In [22]:
def get_recommendations_(title):
    start = time()
    # Get the index of the movie that matches the title
    idx = indices[title]
    cosine_sim = linear_kernel(svd_matrix, svd_matrix[idx].reshape(1, -1))
    
    #print(cosine_sim.shape)
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse= True)
    #print(sim_scores)
    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[:10]
    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]
    end = time()
    print('time :' ,end - start)
    print("__________________________________________________________")
    # Return the top 10 most similar movies
    return metadata['title'].iloc[movie_indices]

In [23]:
get_recommendations_('The Dark Knight Rises')

time : 1.9937283992767334
__________________________________________________________


18252             The Dark Knight Rises
23333                      A Wednesday!
42790    NY77: The Coolest Year in Hell
16888               Hobo with a Shotgun
12071                    Tekkonkinkreet
41656                            Pixels
3613                            Serpico
18035                  Batman: Year One
15511        Batman: Under the Red Hood
21696                      Amsterdamned
Name: title, dtype: object

coclusion : on reste sur le tfidf car il nous donne des résultat meilleur pour la prediction et le svd utilise la matrice pleine alors il prend bceacoup plus de temps 

In [32]:
def get_recommendations_popularity(title):
    start = time()
    # Get the index of the movie that matches the title
    idx = indices[title]
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix[idx])
    #print(cosine_sim.shape)
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse= True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]
    #print(movie_indices)
    get_rec = metadata.iloc[movie_indices]
    get_rec['score'] = get_rec.apply(weighted_rating, axis =1)
    get_rec_data = get_rec.sort_values('score', ascending=False)
    end = time()
    print('time :' ,end - start)
    print("__________________________________________________________")
    # Return the top 10 most similar movies
    return get_rec_data[['title', 'score']]

In [33]:
get_recommendations_popularity('The Dark Knight Rises')

time : 1.3252222537994385
__________________________________________________________


<ipython-input-32-52f23edfddc9>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  get_rec['score'] = get_rec.apply(weighted_rating, axis =1)


,title,score
12481,The Dark Knight,8.265477
19792,"Batman: The Dark Knight Returns, Part 1",7.115637
15511,Batman: Under the Red Hood,7.087743
585,Batman,6.904084
3095,Batman: Mask of the Phantasm,6.645802
9230,Batman Beyond: Return of the Joker,6.534978
18035,Batman: Year One,6.528706
1328,Batman Returns,6.515816
21194,Batman Unmasked: The Psychology of the Dark Kn...,5.846967
150,Batman Forever,5.239617
